In [1]:
import pandas as pd

# Downloading Data

In [47]:
import json
from urllib.request import urlopen

In [51]:
data = []
offset = 0
limit = 10000
resource_id = "PCA_202108"
total = 1


while offset < total:
    # Form the url using offset, limit, resource_id
    url = f"https://opendata.nhsbsa.net/api/3/action/datastore_search?resource_id={resource_id}&limit={limit}&offset={offset}"
    # Openning the url and reading the data
    fileobj = urlopen(url)
    data_json = fileobj.read()
    # Parsing the data to convert from json to python dictionary
    data_dict = json.loads(data_json)
    # Appending the data from this url read to the "data" list
    data += data_dict["result"]["records"]

    offset += limit

    if total == 1:
        total = data_dict["result"]["total"]

IncompleteRead: IncompleteRead(219174 bytes read, 9013509 more expected)

In [ ]:
len(data)

In [43]:
pd.DataFrame.from_records(data_dict["result"]["records"])

,BNF_PARAGRAPH_CODE,BNF_CHAPTER,UNIT_OF_MEASURE,YEAR_MONTH,SNOMED_CODE,REGION_CODE,PREP_CLASS,SUPPLIER_NAME,TOTAL_QUANTITY,DISPENSER_ACCOUNT_TYPE,...,PRESCRIBED_PREP_CLASS,BNF_PARAGRAPH,BNF_CHAPTER_CODE,REGION_NAME,ITEMS,BNF_CHEMICAL_SUBSTANCE,BNF_CHEMICAL_SUBSTANCE_CODE,NIC,BNF_PRESENTATION_NAME,STP_CODE
0,10101,Gastro-Intestinal System,ml,202108,14609911000001108,Y60,1,,400.0,English Pharmacy,...,1,Antacids and simeticone,1,MIDLANDS,2,Magnesium trisilicate,0101010J0,3.30,Magnesium trisilicate oral suspension,QGH
1,10101,Gastro-Intestinal System,ml,202108,14609911000001108,Y63,1,,1600.0,English Pharmacy,...,1,Antacids and simeticone,1,NORTH EAST AND YORKSHIRE,3,Magnesium trisilicate,0101010J0,13.20,Magnesium trisilicate oral suspension,QF7
2,10101,Gastro-Intestinal System,ml,202108,14609911000001108,Y60,1,,1800.0,English Pharmacy,...,1,Antacids and simeticone,1,MIDLANDS,2,Magnesium trisilicate,0101010J0,14.86,Magnesium trisilicate oral suspension,QWU
3,10101,Gastro-Intestinal System,ml,202108,14609911000001108,Y59,1,,800.0,English Pharmacy,...,1,Antacids and simeticone,1,SOUTH EAST,2,Magnesium trisilicate,0101010J0,6.60,Magnesium trisilicate oral suspension,QU9
4,10101,Gastro-Intestinal System,ml,202108,14609911000001108,Y58,1,,800.0,English Pharmacy,...,1,Antacids and simeticone,1,SOUTH WEST,1,Magnesium trisilicate,0101010J0,6.60,Magnesium trisilicate oral suspension,QOX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,20400,Cardiovascular System,tablet,202108,318607008,Y56,1,,63613.0,English Pharmacy,...,1,Beta-adrenoceptor blocking drugs,2,LONDON,2373,Bisoprolol fumarate,0204000H0,2686.33,Bisoprolol 7.5mg tablets,QRV
9996,20400,Cardiovascular System,tablet,202108,318607008,Y61,1,,2212.0,English Dispensing Doctor,...,1,Beta-adrenoceptor blocking drugs,2,EAST OF ENGLAND,85,Bisoprolol fumarate,0204000H0,93.28,Bisoprolol 7.5mg tablets,QM7
9997,20400,Cardiovascular System,tablet,202108,318607008,Y60,1,,20577.0,English Pharmacy,...,1,Beta-adrenoceptor blocking drugs,2,MIDLANDS,772,Bisoprolol fumarate,0204000H0,867.59,Bisoprolol 7.5mg tablets,QPM
9998,20400,Cardiovascular System,tablet,202108,318607008,Y60,1,,2415.0,English Dispensing Doctor,...,1,Beta-adrenoceptor blocking drugs,2,MIDLANDS,111,Bisoprolol fumarate,0204000H0,101.94,Bisoprolol 7.5mg tablets,QK1


# Processing Data

## Import all raw csv

In [2]:
import glob

In [3]:
raw_data_paths = glob.glob("./data/raw_data/*.csv")

# Using python's list comprehension, read each csv from the above paths
raw_data = pd.concat([pd.read_csv(raw_data_path) for raw_data_path in raw_data_paths])
raw_data.head(10)

ValueError: No objects to concatenate

## Monthly Filtered Data

Idea: filter raw data with the file format `./data/raw_data/pca_YYYYMM.csv` and save into `./data/filtered_data/pca_YYYYMM.csv`

In [74]:
def filter_raw_data(raw_df):
    return raw_df[["YEAR_MONTH", "BNF_PRESENTATION_CODE"]]

In [79]:
filtered_data_paths = glob.glob("./data/filtered_data/*.csv")
filtered_data_filenames = set(
    [filtered_data_path.split("/")[-1] for filtered_data_path in filtered_data_paths]
)

raw_data_paths = glob.glob("./data/raw_data/*.csv")

for raw_data_path in raw_data_paths:
    raw_data_filename = raw_data_path.split("/")[-1]
    if raw_data_filename not in filtered_data_filenames:
        print(f"Processing {raw_data_filename}")
        raw_df = pd.read_csv(raw_data_path)
        filtered_df = filter_raw_data(raw_df)
        output_path = f"./data/filtered_data/{raw_data_filename}"
        filtered_df.to_csv(output_path, index=False)

## Product Info

In [58]:
# Selecting a subset of columns
filtered_df = raw_data[
    [
        "BNF_PRESENTATION_CODE",
        "BNF_PRESENTATION_NAME",
        "SNOMED_CODE",
        "GENERIC_BNF_EQUIVALENT_CODE",
        "GENERIC_BNF_EQUIVALENT_NAME",
        "BNF_CHEMICAL_SUBSTANCE_CODE",
        "BNF_CHEMICAL_SUBSTANCE",
        "BNF_PARAGRAPH_CODE",
        "BNF_PARAGRAPH",
        "BNF_SECTION_CODE",
        "BNF_SECTION",
        "BNF_CHAPTER_CODE",
        "BNF_CHAPTER",
        "PREP_CLASS",
        "PRESCRIBED_PREP_CLASS",
    ]
]

In [59]:
# Drop duplicates by BNF_PRESENTATION_CODE
product_info_df = filtered_df.drop_duplicates(subset=["BNF_PRESENTATION_CODE"])

In [60]:
# Export filtered dataframe to csv
product_info_df.to_csv("./data/product_info.csv", index=False)

In [61]:
product_info_df

,BNF_PRESENTATION_CODE,BNF_PRESENTATION_NAME,SNOMED_CODE,GENERIC_BNF_EQUIVALENT_CODE,GENERIC_BNF_EQUIVALENT_NAME,BNF_CHEMICAL_SUBSTANCE_CODE,BNF_CHEMICAL_SUBSTANCE,BNF_PARAGRAPH_CODE,BNF_PARAGRAPH,BNF_SECTION_CODE,BNF_SECTION,BNF_CHAPTER_CODE,BNF_CHAPTER,PREP_CLASS,PRESCRIBED_PREP_CLASS
0,21380000119,KerraPro strip 50 x 2.5 x 0.3cm,2.168431e+16,21380000119,KerraPro strip 50 x 2.5 x 0.3cm,2138,Low Friction Products,2138,Low Friction Products,2138,Low Friction Products,21,Appliances,4,4
1,0403040X0BBACAL,Zispin SolTab 15mg orodispersible tablets,5.608811e+15,0403040X0AAALAL,Mirtazapine 15mg orodispersible tablets,0403040X0,Mirtazapine,40304,Other antidepressant drugs,403,Antidepressant drugs,4,Central Nervous System,3,3
2,090402000BDBGAU,Aymes ActaSolve Delight powder 57g sachets lemon,3.798721e+16,090402000AAAUAU,Generic Aymes ActaSolve Delight powder 57g sac...,090402000,Enteral nutrition,90402,Enteral nutrition,904,Oral nutrition,9,Nutrition and Blood,3,3
3,20100000538,Clinipore tape 5cm,4.796711e+15,20100000553,Permeable non-woven synthetic adhesive tape BP...,2010,Surgical Adhesive Tape,2010,Surgical Adhesive Tape,2010,Surgical Adhesive Tape,20,Dressings,4,4
4,20090000140,Cotton stockinette bleached BP 1988 heavyweigh...,5.885110e+14,20090000140,Cotton stockinette bleached BP 1988 heavyweigh...,2009,Stockinette,2009,Stockinette,2009,Stockinette,20,Dressings,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571731,0406000M0BBADAD,Kytril 2mg tablets,4.722211e+15,0406000M0AAADAD,Granisetron 2mg tablets,0406000M0,Granisetron hydrochloride,40600,Drugs used in nausea and vertigo,406,Drugs used in nausea and vertigo,4,Central Nervous System,3,3
572191,0504010F0BCAAAA,Malarivon 80mg/5ml syrup,4.555611e+15,0504010F0AAAAAA,Chloroquine phosphate 80mg/5ml oral solution,0504010F0,Chloroquine phosphate,50401,Antimalarials,504,Antiprotozoal drugs,5,Infections,3,2
572354,0407020Q0AAELEL,Morphine sulfate 100mg/50ml solution for infus...,1.524871e+16,0407020Q0AAELEL,Morphine sulfate 100mg/50ml solution for infus...,0407020Q0,Morphine sulfate,40702,Opioid analgesics,407,Analgesics,4,Central Nervous System,5,5
572414,21270001547,Solidea Relax Unisex class 1 (18-21mmHg) below...,2.407191e+16,21270001547,Solidea Relax Unisex class 1 (18-21mmHg) below...,2127,Lymphoedema Garments,2127,Lymphoedema Garments,2127,Lymphoedema Garments,21,Appliances,4,4


One problem with this approach is that it gets slower every month as the raw data increases
    => Change the algorithm such that if there exists an existing product_info.csv, simply use that and the latest monthly raw_data to get the updated product_info.csv